In [1]:
import pandas as pd
from texas_hold_em_utils.card import Card
from texas_hold_em_utils.deck import Deck
from texas_hold_em_utils.game import Game
from texas_hold_em_utils.hands import HandOfTwo

In [1]:
# for a given pair of hole cards and a given flop, what is that hand's probability of winning?
# too many cards to break it down to suited/offsuite, so there are 52 first cards, 51 second cards, 50 third cards, 49 fourth cards, 48 fifth cards... 311 million combinations
52*51*50*49*48

# maybe go with sample win rates instead

311875200

In [4]:
def run_game(hole_cards, flop, player_count):
    game1 = Game(player_count)
    deck = Deck()
    deck.remove(hole_cards[0])
    deck.remove(hole_cards[1])
    deck.remove(flop[0])
    deck.remove(flop[1])
    deck.remove(flop[2])
    game1.deck = deck
    game1.community_cards = flop.copy()
    game1.players[0].hand_of_two = HandOfTwo([hole_cards[0], hole_cards[1]])
    deck.shuffle()
    for i in range(1, player_count):
        game1.players[i].hand_of_two = HandOfTwo([deck.draw(), deck.draw()])
        
    game1.round = 1
    game1.turn()
    game1.river()
    
    winners = game1.determine_round_winners()
    if 0 in [winner.position for winner in winners]:
        return 1 / len(winners)
    return 0
    

In [12]:
# win rates seem low, and vary a lot, need to test/debug
hole_cards = [Card().from_str('A', 'Hearts'), Card().from_str('A', 'Spades')]
flop = [Card().from_str('4', 'Diamonds'), Card().from_str('6', 'Clubs'), Card().from_str('8', 'Spades')]

win_ct = 0.0
for i in range(300):
    win_ct += run_game(hole_cards, flop, 3)
    
print(win_ct / 300)

0.6400069381901365


In [15]:
# now to measure variability for different sample sizes
hole_cards = [Card().from_str('A', 'Hearts'), Card().from_str('J', 'Spades')]
flop = [Card().from_str('4', 'Diamonds'), Card().from_str('6', 'Clubs'), Card().from_str('8', 'Spades')]
df = pd.DataFrame(columns=['sample_size', 'win_rate'])
for sample_size in [15, 30, 60, 120]:
    for i in range(30):
        win_ct = 0.0
        for j in range(sample_size):
            win_ct += run_game(hole_cards, flop, 3)
        df = pd.concat([df, pd.DataFrame([[sample_size, win_ct / sample_size]], columns=['sample_size', 'win_rate'])])

C:\Users\amarl\AppData\Local\Temp\ipykernel_34644\4087445459.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([[sample_size, win_ct / sample_size]], columns=['sample_size', 'win_rate'])])


In [17]:
df.to_csv('data/win_rate_variability.csv', index=False)

In [21]:
summary = df.groupby('sample_size', as_index=False).mean()
summary['std_dev'] = df.groupby('sample_size', as_index=False).std()['win_rate']
summary['ci_95_low'] = summary['win_rate'] - 1.96 * summary['std_dev'] / summary['sample_size']**0.5
summary['ci_95_high'] = summary['win_rate'] + 1.96 * summary['std_dev'] / summary['sample_size']**0.5

In [22]:
summary

,sample_size,win_rate,std_dev,ci_95_low,ci_95_high
0,15,0.273333,0.132005,0.20653,0.340137
1,30,0.282223,0.071910,0.25649,0.307955
2,60,0.273889,0.058425,0.259105,0.288673
3,120,0.265278,0.042384,0.257694,0.272861


: 